# Assessing word embeddings for improving OCR accuracy

I will work on this project with Adnan Fazlinovic. 

Our idea is to compare how different embeddings affect OCR accuracy, similar to the approach taken in this article: https://medium.com/states-title/using-nlp-bert-to-improve-ocr-accuracy-385c98ae174c (Links to an external site.)

We would like to try some embeddings we have learned about in the course, such as CBoW, word2vec, FastText, ELMo, BERT. If we have more time and find other interesting representations online we will evaluate them as well. The idea is to find a baseline OCR accuracy and then exploring if, and how much, this accuracy can be improved by applying word embeddings to the incorrectly scanned words. 

- Probably use a synthetic dataset (read in europarl corpus and corrupt it a bit)
- Richard suggests corrupting it in ways that often happen in OCR, like rn <-> m, i <-> l, cl <-> d 
    - (https://scribenet.com/articles/2016/03/04/how-to-get-the-most-out-of-ocr)
- I think he means we should use a spellchecker to find misspelled words, and check with NER that they are not a name
- Then maybe we don't throw out the misspelled word, it can be useful

It feels like we're mostly making a spell/grammar checking model - that's fine

- Richard thinks that character level embeddings can be useful here
- Evaluation of performance: see what people usually use within OCR. Some ideas:

    - https://www.aclweb.org/anthology/I17-1101.pdf
    - https://loicbarrault.github.io/papers/afli_cicling2015.pdf

In [1]:
import re
import spacy
import numpy as np
from enchant.checker import SpellChecker

spell = SpellChecker("en-UK")
nlp = spacy.load('en_core_web_sm')

#import nltk
#from nltk.corpus import stopwords
#stop = stopwords.words('english')

## Try out spellchecker

In [34]:
spell.check("cat's")

True

In [56]:
spell.suggest("spel")

['spiel',
 'spelt',
 'spell',
 'seel',
 'spec',
 'sped',
 'spew',
 'Opel',
 'sp el',
 'sp-el']

In [70]:
import enchant, difflib
word="prblm"

# example - is this a good way to get most relevant word from enchant.spellcheck()?

dict,max = {},0
a = set(spell.suggest(word))

for b in a:

    tmp = difflib.SequenceMatcher(None, word, b).ratio();
    dict[tmp] = b
    
    if tmp > max:
        max = tmp

print(dict[max])

problem
problem


In [64]:
b

'princedom'

# Read in and corrupt the corpus

Character elision is when letter pairs and individual letters are confused by the software. These types of errors occur any time pairs of letters are shaped similarly to other letters. Six common pairs are:

rn <-> m, cl <-> d, vv <-> w, ol <-> d, li <-> h, nn <-> m

In [2]:
elisionArray = []
elisionArray.append(['rn', 'm'])
elisionArray.append(['ol', 'd'])
elisionArray.append(['cl', 'd'])
elisionArray.append(['vv', 'w'])
elisionArray.append(['li', 'h'])
elisionArray.append(['nn', 'm'])
elisionArray = np.array(elisionArray)

## An example of how the character elision will be handled

In [108]:
# A test sentence that contains a lot of 'rn' and 'li'
line = "I smilingly scorn this little barn with my lilac yarn and delicious fern"
print(line)

new_line = line
elision_prob = 0.5

# Randomize search order, since some letter pairs overlap 
elisionArray = np.random.permutation(elisionArray)
for pair in elisionArray:
    
    n_errors = 0
    
    for m in re.finditer(pair[0], new_line):
        
        rd = np.random.rand(1)
        if rd < elision_prob:
            
            # Note that the position can't be used later since the line changes length!
            print("--> Replaced ", pair[0], " at position ", m.start(), "")
            
            tmp = list(new_line)
            tmp[m.start()-n_errors:m.end()-n_errors] = "%%"
            new_line = "".join(tmp)
            new_line = new_line.replace("%%", pair[1])
            
            print(new_line)

            # count number of replacements
            n_errors += 1
            
print('-'*80)

# Save location and correct spelling for the corrupted words

line = list(line.split())
new_line = list(new_line.split())

total_errors = 0
ground_truth = []

# This will be the index of a document in the corpus
doc_num = 0     

# This contains the ground truth for each document 
tmp = []

for j in range(len(line)):
    if line[j] != new_line[j]:
        
        total_errors += 1
        tmp.append((j, line[j]))
        
ground_truth.append((doc_num, tmp))
print("Total errors: ", total_errors)
print(ground_truth)

I smilingly scorn this little barn with my lilac yarn and delicious fern
--> Replaced  rn  at position  15 
I smilingly scom this little barn with my lilac yarn and delicious fern
--> Replaced  rn  at position  32 
I smilingly scom this little bam with my lilac yarn and delicious fern
--> Replaced  rn  at position  51 
I smilingly scom this little bam with my lilac yam and delicious fern
--> Replaced  rn  at position  70 
I smilingly scom this little bam with my lilac yam and delicious fem
--> Replaced  li  at position  5 
I smihngly scom this little bam with my lilac yam and delicious fem
--------------------------------------------------------------------------------
Total errors:  5
[(0, [(1, 'smilingly'), (2, 'scorn'), (5, 'barn'), (9, 'yarn'), (12, 'fern')])]


## Apply character elision to the Europarl data

### Function definitions

In [3]:
def check_line_errors(line, new_line, total_errors):
    
    """
    Check for errors between gold standard document and "OCR" document.
    """

    line = list(line.split())
    new_line = list(new_line.split()) 
    truth = []

    for j in range(len(line)):
        if line[j] != new_line[j]:

            total_errors += 1
            truth.append((j, line[j]))
    
    return truth, total_errors

In [29]:
def read_data(corpus_file, corpus_encoding, max_lines, elisionArray, elision_prob):
    
    """
    Read in ground truth version of the text, and a (synthetic) corrupted OCR dataset.
    """
    
    total_errors = 0
    ground_truth = []
    elision_errors = []
    corrupted_data = []
    
    with open(corpus_file, encoding = corpus_encoding) as f:
        
        for d, line in enumerate(f):
        
            if d == max_lines:
                break
        
            ground_truth.append(line) 
            new_line = line
            
            # Randomize search order, since some letter pairs overlap 
            elisionArray = np.random.permutation(elisionArray)
            for pair in elisionArray:
                
                # Count number of times each letter pair has been corrupted (since this changes the line length)
                n_errors = 0 
                
                for m in re.finditer(pair[0], new_line):
                    
                    rd = np.random.rand(1)
                    if rd < elision_prob:
                        
                        # Replace the letter pair and convert to a new line
                        tmp = list(new_line)
                        tmp[m.start()-n_errors:m.end()-n_errors] = "%%"
                        new_line = "".join(tmp)
                        new_line = new_line.replace("%%", pair[1])

                        # count number of replacements
                        n_errors += 1
                     
            corrupted_data.append(new_line)
            line_truth, total_errors = check_line_errors(line, new_line, total_errors)
            if len(line_truth) > 0:
                elision_errors.append((d, line_truth))
            
    return ground_truth, corrupted_data, elision_errors, total_errors

In [54]:
def identify_spelling_errors(data, ignore):

    """
    Given a dataset and a list of characters to ignore, find misspelled words that are _not_ names.
    """
    
    n_misspelled = 0
    spelling_errors = []
    spelling_corrected = []

    for d, line in enumerate(data):

        words = line.split()
        tmp_1 = []
        tmp_2 = []
        
        for i, word in enumerate(words):

            if not word in ignore and not spell.check(word):

                # Apply nlp pipeline, check if this "misspelled word" is a name
                result = nlp(line, disable = ['tagger', 'parser'])
                is_name = False

                for entity in result.ents:
                    # Ignore "misspelled" names of people, places, organisations - and numbers
                    if entity.label_ in  ["PERSON", "NORP", "GPE", "ORG", "CARDINAL"] and entity.text.find(word) > -1:
                        is_name = True

                if not is_name:

                    n_misspelled += 1

                    # Note down word and position
                    tmp_1.append((i, word))
    
        
        if len(tmp_1) > 0:
            
            spelling_errors.append((d, tmp_1))
            spelling_corrected.append((d, tmp_2))

    return spelling_errors, n_misspelled

### Run the code

In [46]:
file = r"C:\Users\saran\OneDrive\Dokument\GitHub\NLP\Project5\europarl.txt"
encoding = "utf-8"

# Probability that a letter pair from our list will be confused if it is seen in the text
elision_prob = 0.1
max_lines = 100

np.random.seed(0)
ground_truth, data, elision_errors, n_errors = read_data(file, encoding, max_lines, elisionArray, elision_prob)

In [47]:
# Words and characters for the spellchecker to ignore
ignore = [",", ".", '"', "(", ")", "-", "'", "!", "?", ":", ";", "/", "n't", "'s", "'m", "%", "--", "___LANGCODE___"]
spelling_errors, n_misspelled = identify_spelling_errors(data, ignore)

famihes
selfemployed
apphcation
renationalise
ultra-hberal
pohcy
black-marketeering
concems
hke
pditics
Solbes
trialogue


In [48]:
elision_errors

[(15, [(8, 'clear')]),
 (18, [(50, 'families')]),
 (21, [(23, 'application')]),
 (43, [(13, 'ultra-liberal')]),
 (48, [(6, 'policy')]),
 (72, [(1, 'concerns')]),
 (80, [(3, 'like')]),
 (81, [(12, 'politics')])]

In [49]:
spelling_errors

[(18, [(50, 'famihes')]),
 (19, [(3, 'selfemployed')]),
 (21, [(23, 'apphcation')]),
 (30, [(13, 'renationalise')]),
 (43, [(13, 'ultra-hberal')]),
 (48, [(6, 'pohcy')]),
 (59, [(17, 'black-marketeering')]),
 (72, [(1, 'concems')]),
 (80, [(3, 'hke')]),
 (81, [(12, 'pditics'), (28, 'Solbes')]),
 (83, [(30, 'trialogue')])]

In [43]:
print("Number of identified misspelled words: ", n_misspelled)
print("Number of synthetic errors: ", n_errors)

Number of identified misspelled words:  12
Number of synthetic errors:  8


# Compute baseline accuracy using spellcheck suggestions

baseline: use top suggested word from enchant spellchecker

so I guess we create a list of misspelled words just like doc_errors, apply the suggestion for each, and see if we get closer to the ground truth or not?

- https://www.aclweb.org/anthology/I17-1101.pdf

[From the article] For evaluation, we use the CER (Character Error Rate) metric as defined in OCR post-correction evaluations:

$$ CER = \frac{S + D + I}{S + D + C} $$

Where S refers to the number of substituted characters in the OCR text (w.r.t. the reference texts), D to the number of deleted characters, I to the number of inserted characters and C to the number of ‘correct’ characters. We use the CER metric when comparing to the baseline system. __Is this useful in the character elision setting? Doesn't look like it__

We want to measure the systems performance per input, rather than per character. We therefore introduce two complementary accuracy-based evaluation metrics, which evaluate on the level of the character window:
- detection accuracy (detAcc) shows the proportion of correctly detected errors and nonerrors in the evaluated set of 20-character strings
- correction accuracy (corrAcc) reflects the ability of the language model to accurately correct corrupted strings without overgenerating and editing non-corrupted strings

These metrics are calculated as follows:

$detAcc = \frac{(TP + TN + incorrectEdit)}{(TP + TN + FP + FN + incorrectEdit)}$

$corrAcc = \frac{(TP + TN)}{(TP + TN + FP + FN + incorrectEdit)}$

- TP: There is an error on the line, which is corrected.
- TN: No error on the line, no correction
- FP: No error on the line, makes a correction anyway
- FN: There is an error on the line, but it's not corrected
- incorrectEdit: Error is identified and corrected, but other words are also corrected which should not be (within the same document). Basically, anything that doesn't fall into the other categories.

- https://loicbarrault.github.io/papers/afli_cicling2015.pdf

They use BLEU. The Bilingual Evaluation Understudy Score, or BLEU for short, is a metric for evaluating a generated sentence to a reference sentence. A perfect match results in a score of 1.0, whereas a perfect mismatch results in a score of 0.0

Also, they use WER (word error rate). To get the WER, start by adding up the substitutions, insertions, and deletions that occur in a sequence of recognized words. Divide that number by the total number of words originally spoken. The result is the WER. To put it in a simple formula, Word Error Rate = (Substitutions + Insertions + Deletions) / Number of Words Spoken

But how do you add up those factors? Let’s look at each one:

- A substitution occurs when a word gets replaced (for example, “noose” is transcribed as “moose”)
- An insertion is when a word is added that wasn’t said (for example, “SAT” becomes “essay tea”)
- A deletion happens when a word is left out of the transcript completely (for example, “turn it around” becomes “turn around”)

Since we don't consider insertions and deletions, WER doesn't seem very relevant either. 

In [53]:
# From one of the articles
true_line = "n oven for 15 minute"

# True positive case
line_in  = "n o5en for 15 minute"
line_out = "n oven for 15 minute"

# True negative case
line_in  = "n oven for 15 minute"
line_out = "n oven for 15 minute"

# False positive case
line_in  = "n oven for 15 minute"
line_out = "n oven for 15 m1nute"

# False negative case
line_in  = "n o5en for 15 minute"
line_out = "n o5en for 15 minute"

# Incorrect edit case
line_in  = "n o5en for 15 minute"
line_out = "n oven for 15 m1nute"

# Try to fill in missing words using BERT 
(like in the article)

# Try using bigrams 
This could catch when an incorrectly scanned word becomes another, correctly spelled word: for example, God -> Cod. The spell checker will not react to this, but it should create a strange bigram

# Try some other word embeddings

two approaches: either throw away the misspelled word and try to fill it in from context, or use character level embeddings 

# Try some character level embeddings